In [ ]:
import csv
from matplotlib import pyplot as plt
from matplotlib import cm
import numpy as np
from scipy.interpolate import make_interp_spline, BSpline, make_lsq_spline
from scipy.optimize import curve_fit
from curlyBrace import curlyBrace

# Colormap: paired
# Extract colors from colormap, to use at will
colormap = cm.get_cmap('Paired')
colors = [colormap(i) for i in np.linspace(0, 1, 12)]

# Define fitting function
def fitting_exponential(x, y0, v0, k):
    return y0 - (v0/k) * (1 - np.exp(-k*x))

def fitting_sigmoid(x, a, b, c, d):
    return d + (a - d)/(1 + np.power(x / c, b))

In [ ]:
# --- Setup data for FA Dehydrogenation kinetics ---

FA_time = dict([("Co-1", []), ("Co-2", []), ("Co-3", [])])
FA_yield = dict([("Co-1", []), ("Co-2", []), ("Co-3", [])])

with open('data/FA_Dehydrogenation.csv', 'r') as data_file:
    plots = csv.reader(data_file, delimiter=';')
    next(plots)
    for row in plots:
        FA_time['Co-1'].append(row[0])
        FA_yield['Co-1'].append(row[1])
        FA_time['Co-2'].append(row[0])
        FA_yield['Co-2'].append(row[2])
        FA_time['Co-3'].append(row[0])
        FA_yield['Co-3'].append(row[3])

# Cast str to float
FA_yield = {j:[float(i) for i in FA_yield[j] if i] for j in FA_yield}
FA_time = {j:[float(i) for i in FA_time[j] if i] for j in FA_time}

# Create smooth lines on data
# 300 represents number of points to make between T.min and T.max
time_smooth = np.linspace(0, 60, 300) 
smooth_data = dict([("Co-1", []), ("Co-2", []), ("Co-3", []), ("Co-3-reloading", [])])

# Spline interpolation
# spl = make_interp_spline(FA_time['Co-1'], FA_yield['Co-1'], k=3)  # type: BSpline
# smooth_data["Co-1"] = spl(time_smooth)
spl = make_interp_spline(FA_time['Co-2'], FA_yield['Co-2'], k=3)  # type: BSpline
smooth_data["Co-2"] = spl(time_smooth)
# spl = make_interp_spline(FA_time['Co-3'], FA_yield['Co-3'], k=3)  # type: BSpline
# smooth_data["Co-3"] = spl(time_smooth)

# Exponential fitting
popt, _ = curve_fit(fitting_exponential, FA_time['Co-1'], FA_yield['Co-1'])
smooth_data["Co-1"] = fitting_exponential(time_smooth, *popt)
# popt, _ = curve_fit(fitting_exponential, FA_time['Co-2'], FA_yield['Co-2'])
# smooth_data["Co-2"] = fitting_exponential(time_smooth, *popt)
popt, _ = curve_fit(fitting_exponential, FA_time['Co-3'], FA_yield['Co-3'])
smooth_data["Co-3"] = fitting_exponential(time_smooth, *popt)

# Testing lsq data:
# knots = np.linspace(FA_time['Co-3'][0], FA_time['Co-3'][-1], 3)
# knots_t = np.r_[(FA_time['Co-3'][0],)*3, knots, (FA_time['Co-3'][-1],)*3]
# spl = make_lsq_spline(FA_time['Co-3'], FA_yield['Co-3'], t=knots_t, k=3)  # type: BSpline
# smooth_data["Co-3"] = spl(time_smooth)

# --- Starting plotting --- #
# Comparison of catalysts
axis = plt.gca()
axis.set_axisbelow(True)
axis.set_xlabel("Time (min)")
axis.set_xlim(0, 20.5)
axis.set_ylim(-5, 100)
axis.set_ylabel("Gas volume (mL)")

# If secondary axis is required (for yield for instance)
# axis2 = axis.twinx()
# axis2.set_ylabel("Yield ($\%$)")
# axis2.set_ylim(-5, 100)

# Plot each dataset plus smoothing with selected colors.
# Zorder value is used to put it in front of other lines in the graph
plt.plot(FA_time['Co-1'], FA_yield['Co-1'],'o', zorder=2.6, color=colors[6])
plt.plot(time_smooth, smooth_data['Co-1'], '-', zorder=2.8, label="Co-1", color=colors[7])
plt.plot(FA_time['Co-2'], FA_yield['Co-2'],'o', zorder=2.6, color=colors[0])
plt.plot(time_smooth, smooth_data['Co-2'], '-', zorder=2.8, label="Co-2", color=colors[1])
plt.plot(FA_time['Co-3'], FA_yield['Co-3'],'o', zorder=2.6, color=colors[2])
plt.plot(time_smooth, smooth_data['Co-3'], '-', zorder=2.8, label="Co-3", color=colors[3])
plt.legend(fontsize='small')

plt.tight_layout()
plt.savefig("data/FA_Dehydrogenation_Catalysts.png", dpi=300)

In [ ]:
# --- FA Reloading on Co-3 - Data setup ---

refill_time = dict([("refill-1", []), ("refill-2", []), ("refill-3", [])])
refill_yield = dict([("refill-1", []), ("refill-2", []), ("refill-3", [])])

refill_time_crude = []
refill_yield_crude = []
refill_yield_total_crude = []

with open("data/FA_Dehydrogenation_refill.csv") as data_file:
    plots = csv.reader(data_file, delimiter=';')
    next(plots)
    for row in plots:
        refill_time_crude.append(float(row[0]))
        refill_yield_crude.append(float(row[1]))
        refill_yield_total_crude.append(float(row[2]))

refill_time["refill-1"] = refill_time_crude[0:8]
refill_time["refill-2"] = refill_time_crude[8:17]
refill_time["refill-3"] = refill_time_crude[17:]
# Change below to get new yields and not total.
refill_yield["refill-1"] = refill_yield_total_crude[0:8]
refill_yield["refill-2"] = refill_yield_total_crude[8:17]
refill_yield["refill-3"] = refill_yield_total_crude[17:]

# Data fitting part per part
time_smooth = dict([("refill-1", np.linspace(refill_time["refill-1"][0], refill_time["refill-1"][-1], 300)),
                    ("refill-2", np.linspace(refill_time["refill-2"][0], refill_time["refill-2"][-1], 300)),
                    ("refill-3", np.linspace(refill_time["refill-3"][0], refill_time["refill-3"][-1], 300)),
                   ])
popt, _ = curve_fit(fitting_exponential, refill_time["refill-1"], refill_yield['refill-1'])
smooth_data["refill-1"] = fitting_exponential(time_smooth["refill-1"], *popt)
popt, _ = curve_fit(fitting_sigmoid, refill_time["refill-2"], refill_yield['refill-2'], p0=[-200000000, 18, 26, 200])
smooth_data["refill-2"] = fitting_sigmoid(time_smooth["refill-2"], *popt)
popt, _ = curve_fit(fitting_sigmoid, refill_time["refill-3"], refill_yield['refill-3'], p0=[-100000000, 8, 20, 220])
smooth_data["refill-3"] = fitting_sigmoid(time_smooth["refill-3"], *popt)

# Plotting
axis = plt.gca()
axis.set_axisbelow(True)
axis.set_xlabel("Time (min)")
axis.set_xlim(0, 360)
axis.set_ylim(-5, 250)
axis.set_ylabel("Gas volume (mL)")
axis.set_xticks(np.arange(0, 361, step=30))

plt.plot(refill_time["refill-1"], refill_yield["refill-1"], 'o', zorder=2.6, color=colors[2])
plt.plot(refill_time["refill-2"], refill_yield["refill-2"], 'o', zorder=2.6, color=colors[2])
plt.plot(refill_time["refill-3"], refill_yield["refill-3"], 'o', zorder=2.6, color=colors[2])
plt.plot(time_smooth["refill-1"], smooth_data["refill-1"], '-', zorder=2.8, color=colors[3])
plt.plot(time_smooth["refill-2"], smooth_data["refill-2"], '-', zorder=2.8, color=colors[3])
plt.plot(time_smooth["refill-3"], smooth_data["refill-3"], '-', zorder=2.8, color=colors[3])

plt.arrow(60, 50, 0, 30, width=0.5, head_width=4, color=colors[1], fill=True)
plt.text(60, 40, s="FA Refill", horizontalalignment='center', verticalalignment='center')
plt.arrow(120, 120, 0, 30, width=0.5, head_width=4, color=colors[1], fill=True)
plt.text(120, 110, s="FA Refill", horizontalalignment='center', verticalalignment='center')

curlyBrace(plt.gcf(), axis, [60,0], [120,0], 0.05, bool_auto=True, str_text="Refill 1", color='r', lw=2, int_line_num=1)

plt.tight_layout()
plt.savefig("data/FA_Dehydrogenation_Reloading.png", dpi=300)